# Build UNET Model

In [31]:
import torch
import torch.nn as nn

In [32]:
def crop_img(old_tensor, current_tensor):
  current_size = current_tensor.size()[2]#height
  old_size = old_tensor.size()[2]
  cropped_pixel = (old_size - current_size) //2
  # print(cropped_pixel)
  return old_tensor[:, :, cropped_pixel:old_size-cropped_pixel, cropped_pixel:old_size-cropped_pixel]

In [33]:
class DoubleConv(nn.Module):
    def __init__(self, in_channels, out_channels):
        super(DoubleConv, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
            nn.Conv2d(out_channels, out_channels, 3, 1, 1, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=True),
        )

    def forward(self, x):
        return self.conv(x)

In [34]:
class UNET(nn.Module):
    def __init__(
            self, in_channels=1, out_channels=2, features=[64, 128, 256, 512],
    ):
        super(UNET, self).__init__()
        self.up_sampling = nn.ModuleList()
        self.down_sampling = nn.ModuleList()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)

        # Down sampling part
        for feature in features:#small the large image
            self.down_sampling.append(DoubleConv(in_channels, feature))
            in_channels = feature
        self.base_layer = DoubleConv(features[-1], features[-1]*2)

        # Up sampling part
        for feature in reversed(features):#large the small image
            self.up_sampling.append(
                nn.ConvTranspose2d(
                    feature*2, feature, kernel_size=2, stride=2,
                )
            )
            self.up_sampling.append(DoubleConv(feature*2, feature))
        self.out_conv = nn.Conv2d(features[0], out_channels, kernel_size=1)

    def forward(self, x):
      #encoder path
        outs_doubleconv_downs = []
        for down in self.down_sampling:
            x = down(x)
            outs_doubleconv_downs.append(x)
            x = self.pool(x)
        x = self.base_layer(x)

        #decoder path
        outs_doubleconv_downs = outs_doubleconv_downs[::-1]
        for i in range(0, len(self.up_sampling), 2):
            x = self.up_sampling[i](x)
            out_doubleconv_downs = outs_doubleconv_downs[i//2]

            if x.shape != out_doubleconv_downs.shape:
                x = crop_img(out_doubleconv_downs, x)

            concat_img = torch.cat((out_doubleconv_downs, x), dim=1)
            x = self.up_sampling[i+1](concat_img)

        return self.out_conv(x)